In [ ]:
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
import os

# Set seeds for reproducibility
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

# **Loading the Oxford Flowers 102 Dataset**
dataset, info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)
train_data, test_data = dataset['train'], dataset['test']

# Check dataset info
print(info)
print(f'Train data size: {len(train_data)}, Test data size: {len(test_data)}')

# **Data Preprocessing**
def format_image(image, label):
    image = tf.image.resize(image, (150, 150))  # Resize to 150x150
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image, label

train_data = train_data.map(format_image).batch(32).shuffle(1000)
test_data = test_data.map(format_image).batch(32)

# **Displaying Sample Image**
for image_batch, label_batch in train_data.take(1):
    plt.imshow(image_batch[0])  # Display the first image
    plt.axis('off')
    plt.show()
    print('Label:', label_batch[0].numpy())  # Display the label of the first image

# **Building the Convolutional Neural Network**
model = models.Sequential([
    # First convolutional block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Second convolutional block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Third convolutional block
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Fourth convolutional block
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Fifth convolutional block
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Sixth convolutional block
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Seventh convolutional block
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Eighth convolutional block
    layers.Conv2D(512, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    
    # Flattening the output and adding dense layers
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),  # Adding dropout to reduce overfitting
    layers.Dense(102, activation='softmax')  # 102 classes for Oxford Flowers
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# **Training the Model**
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
history = model.fit(train_data, epochs=20, validation_data=test_data, callbacks=[early_stopping])

# **Evaluating the Model**
test_loss, test_acc = model.evaluate(test_data)
print(f'\nTest accuracy: {test_acc}')

# **Plotting Accuracy and Loss**
def plot_metrics(history):
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

plot_metrics(history)

# **Saving the Model**
# Use a relative path that is not nested too deep for Streamlit deployment
model_save_path = 'https://github.com/sushmithacv/OxfordFlower102/trained_oxford_flowers_model.h5'
model.save(model_save_path)
print(f"Model saved at {model_save_path}")

# **Loading the Model in Streamlit**
# In your Streamlit app, use error handling to load the model
def load_model_with_error_handling():
    if os.path.exists(model_save_path):
        model = models.load_model(model_save_path)
        print("Model loaded successfully.")
    else:
        print(f"Error: Model file {model_save_path} not found.")
        return None
    return model

# Example usage in Streamlit
# model = load_model_with_error_handling()
